In [ ]:
import os
import pandas as pd

# Specify the folder containing your Excel files
data_folder = "Data"

# Define the column names
col_names = ['Case ID', 'Suspect Product Active Ingredients', 'Reason for Use', 'Reactions', 'Serious', 'Outcomes', 'Sex', 'Patient Age', 'Patient Weight']

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame(columns=col_names)

# Iterate over all Excel files in the folder
for file in os.listdir(data_folder):
    if file.endswith(".xlsx"):  # Check if the file is an Excel file
        file_path = os.path.join(data_folder, file)
        # Read the Excel file and add it to the combined DataFrame
        data = pd.read_excel(file_path, usecols=col_names)  # Load only the specified columns
        combined_data = pd.concat([combined_data, data], ignore_index=True)

# Display the combined dataset
#print(combined_data)

In [ ]:
selected_columns = ['Case ID', 'Suspect Product Active Ingredients', 'Reactions',  'Serious', 'Sex', 'Patient Age', 'Patient Weight']

#df = data[selected_columns]
df = combined_data[selected_columns]

#df.head()
#print(df)

In [ ]:
df['Suspect Product Active Ingredients'] = df['Suspect Product Active Ingredients'].str.split(';')
df_split_temp = df.explode('Suspect Product Active Ingredients', ignore_index=True)

df_split_temp['Reactions'] = df_split_temp['Reactions'].str.split(';')
df_split = df_split_temp.explode('Reactions', ignore_index = True)

#df_split.head()

In [ ]:
df_multi = pd.get_dummies(df_split, columns = ['Suspect Product Active Ingredients', 'Reactions'], prefix=['Product', 'Reaction'], prefix_sep='_')
reactions = [col for col in df_multi.columns if col.startswith("Reaction_")]
df_reaction = df_multi[reactions]
df_multi = df_multi.groupby('Case ID').max().reset_index()

columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Reactions', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

columns_to_convert = [col for col in df_multi.columns if col.startswith("Product_") or col.startswith("Reaction_")] #not in columns_to_exclude]

df_multi[columns_to_convert] = df_multi[columns_to_convert].astype(int)

df_final = df_multi
#print(df_multi.columns)
#print(df_final)

#df_final.head()

In [ ]:
df_encoded = df_multi.copy()
#print(df_encoded.columns)
#print(df_split['Patient Age'])
df_encoded['Patient Age'] = df_encoded['Patient Age'].astype(str)
df_encoded['Patient Age'] = df_encoded['Patient Age'].str.replace(r'\D+', '', regex=True)
df_encoded['Patient Age'] = pd.to_numeric(df_encoded['Patient Age'], errors='coerce')  # Converts to numeric, sets invalid values to NaN

df_encoded['Patient Weight'] = df_encoded['Patient Weight'].replace('Not Specified', "0 KG")
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].astype(str)
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].str.replace(r'[^\d.]', '', regex=True)
df_encoded['Patient Weight'] = pd.to_numeric(df_encoded['Patient Weight'], errors='coerce')

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

#df_encoded['Suspect Product Active Ingredients'] = label_encoder.fit_transform(df_encoded['Suspect Product Active Ingredients'])
df_encoded['Sex'] = label_encoder.fit_transform(df_encoded['Sex'])
df_encoded['Serious'] = label_encoder.fit_transform(df_encoded['Serious'])
#print(df_encoded.isnull().sum())
df_encoded.dropna(inplace=True)
#print(df_encoded.isnull().sum())
#df_encoded.head()

In [ ]:
# Sum columns that start with "Reaction_"
reaction_columns = df_encoded.filter(like="Reaction_", axis=1)
column_sums = reaction_columns.sum(axis=0)

#print(len(reaction_columns.columns))
# Set the threshold
threshold = 50 #300

# Filter column names with sums greater than the threshold
columns_above_threshold = column_sums[column_sums > threshold].index.tolist()

#print(columns_above_threshold)
#print(len(columns_above_threshold))

filtered_df = df_encoded[df_encoded[columns_above_threshold].sum(axis=1) > 0]

# Print the number of rows before and after filtering
#print(f"Number of rows before filtering: {df_encoded.shape[0]}")
#print(f"Number of rows after removing all-zero rows: {filtered_df.shape[0]}")

In [ ]:
cols_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Serious' ,'Reactions']

feature_cols = [col for col in df_encoded.columns if col not in cols_to_exclude and not col.startswith("Reaction_")]

print(feature_cols)
print(len(feature_cols))

X_serious = df_encoded[feature_cols]
Y_serious = df_encoded.Serious

# Reaction Predictor

feat_cols = ["Serious"]
feat_cols.extend(feature_cols)

print(feat_cols)
print(len(feat_cols))

predict_cols = []#["Serious"]
reaction_cols = columns_above_threshold #[col for col in df_encoded.columns if col.startswith("Reaction_")]
predict_cols.extend(reaction_cols)

print(predict_cols)
print(len(predict_cols))

df_filtered = df_encoded[df_encoded[predict_cols].sum(axis=1) > 0]

X_reaction = df_filtered[feat_cols]
Y_reaction = df_filtered[predict_cols]